### ERNESTO ANDRÉS GONZÁLEZ LOMELÍ

### EXPEDIENTE : 746843

## GINI

El índice de Gini mide la impureza de una partición, es decir, qué tan mezcladas están las clases en un nodo.

- Valor mínimo es 0: nodo puro (todos los datos son de la misma clase).
- Valor máximo (hasta 0.5 o más dependiendo del número de clases), Lo que significa que es nodo completamente mezclado.

**Conexión con las particiones:** Al construir un árbol de decisión, se busca dividir los datos minimizando la impureza. Es decir, la mejor división es aquella que reduce más el Gini total de los nodos hijos.



## Entropía

La entropía mide el grado de desorden o aleatoriedad en un conjunto. Al igual que Gini, se usa para medir impureza, pero es más sensible a distribuciones de clases más balanceadas.

**Conexión con las particiones:** Se utiliza para calcular  cuánta entropía se pierde (o se gana en pureza) al hacer una división.

## Log Loss

Log Loss se usa típicamente en modelos probabilísticos de clasificación como la regresión logística. Penaliza más fuertemente las predicciones incorrectas con alta confianza.

**Conexión con las particiones:** No se usa directamente en árboles como criterio de división, pero es útil para evaluar la calidad probabilística de un modelo. Algunos algoritmos como XGBoost o LightGBM pueden usar variantes de Log Loss para optimizar la clasificación.

## Diferencia entre entropía y log loss

- La entropía se usa en la división de árboles y el log loss en la evaluación de modelos
- La entropía mide la impureza de un conjunto y el log loss la precesión de las probabilidades antes dichas
- El tipo de predicción que tiene el log loss es la probabilidad para cada clase y la entropía la clase dominante
- El log loss penaliza más fuerte los errores de alta confianza

In [1]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

In [2]:
iris = load_iris(as_frame=True)
data = iris.frame
data['target'] = data['target'].astype(int)  
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
condicion = data['petal length (cm)'] < 2.5
grupo_izquierda = data[condicion]
grupo_derecha = data[~condicion]

In [4]:
print(f"Grupo izquierda: {len(grupo_izquierda)}")
print(f"Grupo derecha: {len(grupo_derecha)}")

Grupo izquierda: 50
Grupo derecha: 100


In [5]:
#GINI
def gini_impureza(y):
    clases, counts = np.unique(y, return_counts=True)
    p = counts / counts.sum()
    return 1 - np.sum(p ** 2)


In [6]:
gini_izquierda = gini_impureza(grupo_izquierda['target'])
gini_derecha = gini_impureza(grupo_derecha['target'])

In [7]:
total = len(data)
gini_total = (len(grupo_izquierda) / total) * gini_izquierda + \
             (len(grupo_derecha) / total) * gini_derecha

print(f"GINI izquierda: {gini_izquierda:.4f}")
print(f"GINI derecha: {gini_derecha:.4f}")
print(f"GINI ponderado total: {gini_total:.4f}")

GINI izquierda: 0.0000
GINI derecha: 0.5000
GINI ponderado total: 0.3333


In [8]:
#Entropía
def entropia(y):
    clases, counts = np.unique(y, return_counts=True)
    p = counts / counts.sum()
    return -np.sum(p * np.log2(p))


In [9]:
entropia_izquierda = entropia(grupo_izquierda['target'])
entropia_derecha = entropia(grupo_derecha['target'])

In [10]:
entropia_total = (len(grupo_izquierda) / total) * entropia_izquierda + \
                 (len(grupo_derecha) / total) * entropia_derecha

print(f"Entropía izquierda: {entropia_izquierda:.4f}")
print(f"Entropía derecha: {entropia_derecha:.4f}")
print(f"Entropía ponderada total: {entropia_total:.4f}")

Entropía izquierda: -0.0000
Entropía derecha: 1.0000
Entropía ponderada total: 0.6667


In [11]:
from sklearn.metrics import log_loss

clases = np.unique(data['target'])

def log_loss_simplificado(grupo):
    y_true = grupo['target'].values
    
    p_clases = np.bincount(y_true, minlength=3) / len(y_true)

    y_proba = np.tile(p_clases, (len(y_true), 1))
    return log_loss(y_true, y_proba, labels=clases)


In [12]:
ll_izq = log_loss_simplificado(grupo_izquierda)
ll_der = log_loss_simplificado(grupo_derecha)
ll_total = (len(grupo_izquierda) * ll_izq + len(grupo_derecha) * ll_der) / len(data)

print(f"Log Loss izquierda: {ll_izq:.4f}")
print(f"Log Loss derecha: {ll_der:.4f}")
print(f"Log Loss ponderado total: {ll_total:.4f}")

Log Loss izquierda: 0.0000
Log Loss derecha: 0.6931
Log Loss ponderado total: 0.4621


#### **CONCLUSIONES**

1. GINI:

- El grupo izquierdo tiene GINI = 0.0000, lo que indica que es completamente puro (solo una clase).
- El grupo derecho tiene GINI = 0.5000, lo que indica mezcla de clases.
- El GINI total ponderado fue 0.3333, reflejando que la división logró cierta mejora en pureza

2. Entropía:

- La entropía del grupo izquierdo fue prácticamente 0 (puro)
- La del grupo derecho fue 1.0000, indicando máxima impureza
- La entropía ponderada total fue 0.6667, mayor que el GINI, ya que la entropía es más sensible a clases balanceadas.

3. Log Loss:

- El grupo izquierdo tuvo Log Loss = 0.0000 (perfecta predicción)
- El grupo derecho tuvo Log Loss = 0.6931, valor típico cuando hay incertidumbre entre dos clases.
- El Log Loss total ponderado fue 0.4621, lo que nos indica una pérdida media en la calidad de predicción 

### **Observación**

Escogí la partición petal length < 2.5 porque permite separar perfectamente la clase Setosa del resto, generando un grupo completamente puro. Esto facilita el análisis de los criterios GINI, Entropía y Log Loss, ya que se puede observar claramente cómo reaccionan ante un grupo puro versus uno mixto.